### 安装 LangChain 与 OpenAI


In [1]:
import os
import API_KEYS
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = API_KEYS.AZURE_ENDPOINT
os.environ["OPENAI_API_KEY"] = API_KEYS.AZURE_API_KEY

In [2]:
from langchain.llms import AzureOpenAI

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    deployment_name=API_KEYS.AZURE_DEPLOYMENT_NAME_LLM,
    temperature=0
)

In [3]:
llm("微软公司的创始人是谁？")

'\n\n微软公司的创始人是比尔·盖茨和保罗·艾伦。'

#### Agents: 基于用户输入动态调用 Chain


In [4]:
# pip install google-search-results

os.environ["SERPAPI_API_KEY"] = API_KEYS.SERPAPI_API_KEY

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import AzureOpenAI

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    deployment_name=API_KEYS.AZURE_DEPLOYMENT_NAME_LLM,
    streaming=True,
    temperature=0
)


 # 加载 serpapi 工具
tools = load_tools(["serpapi"])

# 如果搜索完想在计算一下可以这么写
# tools = load_tools(['serpapi', 'llm-math'], llm=llm)

# 如果搜索完想再让他再用python的print做点简单的计算，可以这样写
# tools=load_tools(["serpapi","python_repl"])

# 工具加载后都需要初始化，verbose 参数为 True，会打印全部的执行详情
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [6]:
# 运行 agent
agent.run("明源云是做什么业务的？上市了没有？老板是谁？")



> Entering new AgentExecutor chain...
 I need to find out more information about this company.
Action: Search
Action Input: 明源云
Observation: 明源云，成立于2003年（其前身公司创始于1997年），是国内领先的不动产生态链数字化解决方案服务商，香港联交所主板上市（0909.HK）已经累计为7000多家不动产开发运营 ...
Thought: I now know the company is listed on the Hong Kong Stock Exchange.
Action: Search
Action Input: 明源云老板
Observation: 在2003年，由主席及首席执行官高宇、执行董事陈晓晖和姜海洋，于总部深圳成立“深圳市明源拓展软件科技有限公司”，经营中国内地和全球经营地产生态链数字化服务。
Thought: I now know the company was founded by Gao Yu, Chen Xiaohui, and Jiang Haiyang.
Final Answer: 明源云是一家国内领先的不动产生态链数字化解决方案服务商，香港联交所主板上市（0909.HK），由主席及首席执行官高宇、执行董事陈晓晖和姜海洋创立。

> Finished chain.


'明源云是一家国内领先的不动产生态链数字化解决方案服务商，香港联交所主板上市（0909.HK），由主席及首席执行官高宇、执行董事陈晓晖和姜海洋创立。'

**agent type 几个选项的含义（理解不了也不会影响下面的学习，用多了自然理解了）：**

- zero-shot-react-description: 根据工具的描述和请求内容的来决定使用哪个工具（最常用）
- react-docstore: 使用 ReAct 框架和 docstore 交互, 使用 Search 和 Lookup 工具, 前者用来搜, 后者寻找 term, 举例: Wipipedia 工具
- self-ask-with-search 此代理只使用一个工具: Intermediate Answer, 它会为问题寻找事实答案(指的非 gpt 生成的答案, 而是在网络中,文本中已存在的), 如 Google search API 工具
- conversational-react-description: 为会话设置而设计的代理, 它的 prompt 会被设计的具有会话性, 且还是会使用 ReAct 框架来决定使用来个工具, 并且将过往的会话交互存入内存
